<p style="font-family: Arial; font-size:3.0em; color:purple; font-style:bold">
<br>SHOWING FANAL BB-ANALYSIS RESULTS<br></p>

This notebook illustrates the results got from FANAL 'bb' analysis.

It provides contents of generated DataFrames with some associated plots.

#### Setting general stuff

In [1]:
from IPython.core.display import HTML
css = open('css/style-table.css').read() + open('css/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%matplotlib inline

#### Importings

In [4]:
import os
import sys
import numpy  as np
import tables as tb
import pandas as pd
import matplotlib.pyplot as plt
from   matplotlib.colors import LogNorm

#%matplotlib widget

In [5]:
# Specific IC stuff

import invisible_cities.core.system_of_units  as units

In [6]:
# Specific FANAL stuff

from fanal.containers.events import EventCounter

In [7]:
plt.rcParams["figure.figsize"] = 8, 6
plt.rcParams["font.size"     ] = 13

# LOADING DATA

In [8]:
#results_fname = '/Users/Javi/Development/FANAL/fanal/fanal.tst.h5'

iPATH         = "../data/next100"
#results_fname = iPATH + "/Bi214/fanal/fanal.next100.Bi214.fwhm_05.voxel_3x3x3.h5"
#results_fname = iPATH + "/Bi214/fanal/fanal.next100.Bi214.fwhm_07.voxel_10x10x10.h5"
#results_fname = iPATH + "/Tl208/fanal/fanal.next100.Tl208.fwhm_05.voxel_3x3x3.h5"
#results_fname = iPATH + "/Tl208/fanal/fanal.next100.Tl208.fwhm_07.voxel_10x10x10.h5"
#results_fname = iPATH + "/bb0nu/fanal/fanal.next100.bb0nu.fwhm_05.voxel_3x3x3.h5"
results_fname = iPATH + "/bb0nu/fanal/fanal.next100.bb0nu.fwhm_07.voxel_10x10x10.h5"

In [9]:
config_df  = pd.read_hdf(results_fname, "FANAL" + '/config')
events_df  = pd.read_hdf(results_fname, "FANAL" + '/events')
tracks_df  = pd.read_hdf(results_fname, "FANAL" + '/tracks')
voxels_df  = pd.read_hdf(results_fname, "FANAL" + '/voxels')
results_df = pd.read_hdf(results_fname, "FANAL" + '/results')

KeyError: 'No object named FANAL/events in the file'

In [ ]:
print("Configuration run ...")
config_df

# EVENT counters

In [ ]:
results_df

In [ ]:
print(EventCounter(**results_df.events))

In [ ]:
# Accessing a single value
print(f"Num events that passed the ROI filter: {results_df.loc['roi_filter'].events}")

In [ ]:
results_df.plot.bar(title = 'ANALYSIS RESULTS', rot=45, log=True)

# DataFrames

In [ ]:
evt_cols_to_show = ['mc_energy', 'mc_filter', 'sm_energy', 'energy_filter', 'num_voxels', 'fiduc_filter',
                    'num_tracks', 'track_length', 'track_filter', 'blob1_energy', 'blob2_energy',
                    'blob_filter', 'roi_filter']

In [ ]:
# Showing the desired columns from first events
events_df[evt_cols_to_show].head()

In [ ]:
# Showing just the events that passed the TRACK filter
events_df[events_df.track_filter][evt_cols_to_show]

In [ ]:
# Showing first tracks
tracks_df.head()

In [ ]:
# Showing first voxels
voxels_df.head()

# Some graphics

In [ ]:
max_num_trks = 10
plt.hist(events_df.num_tracks, max_num_trks, [0, max_num_trks])
plt.xlabel('Num Tracks')
plt.ylabel('Num. events')
plt.title('Number of reconstructed tracks')

In [ ]:
# Plotting energy, length and number of voxels of single track events
evts_to_show = events_df[events_df.num_tracks == 1]

fig = plt.figure(figsize = (18, 5))
E_bins, E_range           = 20, [2.42, 2.50]
voxels_bins, voxels_range = 20, [0, 50]
length_bins, length_range = 20, [0, 200]

# First track plots
ax1 = fig.add_subplot(1, 3, 1)
plt.hist(evts_to_show.sm_energy, E_bins, E_range)
plt.title('Track Energy [MeV]')

ax2 = fig.add_subplot(1, 3, 2)
plt.hist(evts_to_show.num_voxels, voxels_bins, voxels_range)
plt.title('Track - Num Voxels')

ax3 = fig.add_subplot(1, 3, 3)
plt.hist(evts_to_show.track_length, length_bins, length_range)
plt.title('Track - Length')

In [ ]:
from fanal.utils.types import XYZ

# Plotting distance between True extrema and Blob positions
# of events passing the track_filter
evts_to_show = events_df[events_df.track_filter]
trks_to_show = tracks_df.loc[evts_to_show.index]

# Getting distances
distances_1 = XYZ(trks_to_show.t_ext1_x, trks_to_show.t_ext1_y, trks_to_show.t_ext1_z). \
              distance(XYZ(trks_to_show.blob1_x, trks_to_show.blob1_y, trks_to_show.blob1_z))

distances_2 = XYZ(trks_to_show.t_ext2_x, trks_to_show.t_ext2_y, trks_to_show.t_ext2_z). \
              distance(XYZ(trks_to_show.blob2_x, trks_to_show.blob2_y, trks_to_show.blob2_z))

distances   = pd.concat([distances_1, distances_2])

# PLOTTING
fig = plt.figure(figsize  = (18, 5))
dist_bins, dist_range     = 20, [0., 50.]

# First distance
ax1 = fig.add_subplot(1, 3, 1)
plt.hist(distances_1, dist_bins, dist_range)
plt.title('tExtreme-Blob 1 distance [mm]')

# Second distance
ax2 = fig.add_subplot(1, 3, 2)
plt.hist(distances_2, dist_bins, dist_range)
plt.title('tExtreme-Blob 2 distance [mm]')

# All distances
ax3 = fig.add_subplot(1, 3, 3)
plt.hist(distances, dist_bins, dist_range)
plt.title('tExtreme-Blob All distance [mm]')


In [ ]:
# Plotting blob energies of events passing the track_filter
evts_to_show = events_df[events_df.track_filter]
trks_to_show = tracks_df.loc[evts_to_show.index]

fig = plt.figure(figsize = (15,12))
hit_bins, hit_range = 30, [0., 220]
E_bins, E_range     = 30, [0., 1.50]

ax1 = fig.add_subplot(2, 2, 1)
plt.hist(trks_to_show.blob1_num_hits, hit_bins, hit_range)
plt.xlabel('Number of hits')
plt.title(f'Blob1 Num hits')

ax2 = fig.add_subplot(2, 2, 2)
plt.hist(trks_to_show.blob2_num_hits, hit_bins, hit_range)
plt.xlabel('Number of hits')
plt.title(f'Blob2 Num hits')

ax3 = fig.add_subplot(2, 2, 3)
plt.hist(evts_to_show.blob1_energy, E_bins, E_range)
plt.xlabel('Energy [MeV]')
plt.title(f'Blob1 Energy [MeV]')

ax4 = fig.add_subplot(2, 2, 4)
plt.hist(evts_to_show.blob2_energy, E_bins, E_range)
plt.xlabel('Energy [MeV]')
plt.title(f'Blob2 Energy [MeV]')

fig = plt.figure(figsize = (7,6))
plt.hist2d(evts_to_show.blob1_energy, evts_to_show.blob2_energy, E_bins,
           [E_range, E_range], norm=LogNorm())
plt.xlabel('Highest Blob Energy [MeV]')
plt.ylabel('Lowest Blob Energy [MeV]')
plt.title(f'Blob Energies [MeV]')

In [ ]:
# Plotting event energy of events passing the ROI filter
evts_to_show = events_df[events_df.roi_filter]

E_range = [2.445, 2.470]
E_bins = int((E_range[1] - E_range[0]) / units.keV)

plt.hist(evts_to_show.sm_energy, E_bins, E_range)
plt.xlabel('Event Energy [MeV]')
plt.title((f'Event Energy'))

# Printing and Plotting

#### Show functions

In [ ]:
from fanal.utils.mc_utils    import print_mc_event
from fanal.utils.rec_utils   import print_rec_event
from fanal.utils.plot_utils  import plot_mc_event
from fanal.utils.plot_utils  import plot_rec_event

In [ ]:
def show_event(event_id     : int,
               input_fname  : str,
               output_fname : str,
               event_type   : str,
               print_mc     : bool = True,
               print_rec    : bool = True,
               plot_mc      : bool = True,
               plot_rec     : bool = True
              ):
    print(f"\n\n*** SHOWING EVENT ID: {event_id} ***\n")
    
    if print_mc : print_mc_event (event_id, input_fname, with_hits=False)
    if print_rec: print_rec_event(event_id, output_fname)
    if plot_mc  : plot_mc_event  (event_id, input_fname, event_type)
    if plot_rec : plot_rec_event (event_id, output_fname)

#### Selecting event ids to show

In [ ]:
# Single event
event_ids = 170

# Events that passed the ROI filter
#event_ids = events_df[events_df.roi_filter].index.tolist()

# Events that passed the energy filter but not the fiducial one
#event_ids = events_df[(events_df.energy_filter == True) &
#                      (events_df.fiduc_filter == False)].index.tolist()

# Events with blob oerlap energy > 0
#event_ids = tracks_df[tracks_df.ovlp_energy > 0.].index.get_level_values('event_id').tolist()

print("Event Ids to show:", event_ids)

#### Filenames & Event type

In [ ]:
input_fname  = config_df.loc['input_fname'].value
output_fname = config_df.loc['output_fname'].value

print(f"Input  file name: {input_fname}")
print(f"Output file name: {output_fname}")

event_type = config_df.loc['event_type'].value
print(f"Event type: {event_type}")

#### Showing events

In [ ]:
event_ids = [event_ids] if isinstance(event_ids, int) else event_ids
for event_id in event_ids:
    show_event(event_id,
               input_fname  = input_fname,
               output_fname = output_fname,
               event_type   = event_type,
               print_mc     = False,
               print_rec    = True,
               plot_mc      = False,
               plot_rec     = False)